In [22]:
import csv 
from pathlib import Path
from os import listdir
import pickle

from labeling_utils import load_labels

import numpy as np
from sklearn.metrics import precision_recall_fscore_support


# tags=["Songbird","Water Bird","Insect","Running Water","Rain","Cable","Wind","Vehicle","Aircraft"]
tags=["Songbird","Water Bird","Insect","Running Water","Rain","Cable","Wind","Aircraft"]

from tabulate import tabulate
tag_set=tags[:]


* Count only highest ranking tag (what if both of them exist)
* Bird tag is confusing, it can be also waterbird, how to handle ? Also Animal
* 

In [23]:
#LOAD MODEL predictions
splits_path= Path('/files/scratch/enis/data/nna/labeling/splits/')

labelsbymodelpath=Path('/scratch/enis/data/nna/labeling/deep_labels/')
dbfile = open(labelsbymodelpath / "classifications_all_2.pkl", 'rb')  
modelresults = pickle.load(dbfile) 
dbfile.close() 

In [24]:
# print(len(modelresults))
# aiter=iter(modelresults.keys())
# akey=next(aiter)
# print(akey)
# modelresults[akey]

In [25]:
# LOAD LABELS by human
labelsbyhumanpath=Path('/scratch/enis/data/nna/labeling/results/')
# filter by username
labelsbyhuman=[i for i in listdir(labelsbyhumanpath) if (".csv" in i) ]

humanresults={}
counter=0
for apath in labelsbyhuman:
    with open(labelsbyhumanpath / apath, newline='') as f:
        reader=csv.reader(f)
        for row in reader:
            counter+=1
            humanresults[row[0]]=row[1:]

print("unique files:",len(humanresults),"\ntotal files",counter)

unique files: 1300 
total files 1366


In [26]:
#Join vehicle and Aircraft
for file_name,tagshere in humanresults.items():
#     print(file_name,tagshere)
    
    tagshere=["Aircraft" if tag == "Vehicle" else tag for tag in tags]

In [27]:
histogram={}
for file_name in humanresults:
    place_name=file_name.split("_")[0]
#     print(file_name)
    histogram.setdefault(place_name,0)
    histogram[place_name]+=1

In [28]:
a=list(histogram.items())
a.sort(key=lambda x:x[1],reverse=True)
# a

In [29]:
# load name of the labels
labels=load_labels()

In [30]:
items={}
from IPython.display import Audio,display,HTML
from ipywidgets import Output

items["mp3_output"]=Output()

def play_html_modify(mp3file,items={}):
#     out=items["mp3_output"]
#     displayed=display(HTML("<audio controls  loop autoplay><source src={} type='audio/mpeg'></audio>".format(mp3file)))
    displayed=display(HTML("<audio controls  loop><source src={} type='audio/mpeg'></audio>".format(mp3file)))


In [31]:
def vectorized_y_true(humanresults,tag_set):
    y_true={tag: [None]*len(humanresults) for tag in tag_set}
    for i,tags in enumerate(humanresults.values()):
        # we  only look for tags in tag_set
        for tag in tag_set:
            if tag in tags:
                y_true[tag][i] = 1
            else:
                y_true[tag][i] = 0 
    return y_true

In [32]:
y_true_dict=vectorized_y_true(humanresults,tag_set)


In [33]:
# for i,filename in enumerate(humanresults.keys()):
#     orig,audioop = modelresults[filename]


In [34]:
split_path=Path('/scratch/enis/data/nna/labeling/split_embeddings/')
# filter by username
split_embeds=[i for i in listdir(split_path) ]
raw_embeds =  [i for i in split_embeds if "rawembed" in i]
proc_embeds = [i for i in split_embeds if "_embed" in i]

In [35]:
len(proc_embeds),len(raw_embeds),len(split_embeds)

(10439, 10439, 20878)

In [36]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import roc_auc_score


In [37]:

h = .02  # step size in the mesh

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(probability=True,kernel="linear", C=0.025),
    SVC(probability=True,gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]



In [38]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
    
short_names=['N. Neighbors',
 'Linear SVM',
 'RBF SVM',
 'Gaussian P.',
 'Decision T.',
 'Random F.',
 'NN',
 'AdaBoost',
 'Naive B.',
 'QDA']

def print_result(results):
    headers= ["Tag",]+short_names
    table=[]
#     sample_count=len(next(iter(y_true_dict.values())))
#     print("Total sample:",sample_count,"And threshold is",prob_threshold)
    for tag in (tag_set):
#         positive_count=sum(y_true_dict[tag])
        table.append([tag,*results[tag]])

    print(tabulate(table, headers=headers))

def reverse_results(results):
    results_clf2={}
    for tag,scores_per_Tag2 in results.items():
        scores_per_Tag=scores_per_Tag2[:]
        maximum=max(scores_per_Tag)
        scores_per_Tag=[color.BOLD + x + color.END if x==maximum else x for x in scores_per_Tag]

        for i,name in enumerate(names):
            score=scores_per_Tag[i]
            results_clf2.setdefault(name,[])
            results_clf2[name].append(score)      
    return results_clf2
        

def print_result2(results):
    results_clf=reverse_results(results)
    headers= ["Tag",]+list(tag_set)
    table=[]
#     sample_count=len(next(iter(y_true_dict.values())))
#     print("Total sample:",sample_count,"And threshold is",prob_threshold)
#     for tag in tag_set:
#         aresult=[]
    for i,name in enumerate(names):
#         positive_count=sum(y_true_dict[tag])
        table.append([short_names[i],*results_clf[name]])
    
    print(tabulate(table, headers=headers))


In [39]:
def get_results(X_input,y_true_dict,classifiers,names,tag_set,logs=False):
    results={}
    for tag in (tag_set):
        X_train, X_test, y_train, y_test = train_test_split(
            X_input, y_true_dict[tag], test_size=0.2, random_state=42)
        X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=0.25, random_state=42)

        for name, clf in zip(names, classifiers):
            try:
                clf.fit(X_train, y_train)
                #VAL
#                 y_val_pred=clf.predict_proba(X_val)
#                 score=roc_auc_score(y_val,y_val_pred[:,1:])
                
                # TEST
                y_test_pred=clf.predict_proba(X_test)
                score=roc_auc_score(y_test,y_test_pred[:,1:])
    #             score = clf.score(X_val, y_val)
                score="{:.2}".format(score)

                results.setdefault(tag,[])

                results[tag].append(score)

                if logs : print(tag,name,score) 
            except:
                score="-1"

                results.setdefault(tag,[])

                results[tag].append(score)

                if logs : print(tag,name,score) 
    return results

In [40]:
def map_reduce(X,func_type):
    if func_type=="Average":
        return np.mean(X,axis=1)
    elif func_type=="Concat":
        return np.reshape(X,(-1,1280))
    else:
        raise Exception("ERROR with embed type")

def pick_embed(embed_type):
    # humanresults[proc_embeds[0].replace("_embed.npy",".mp3")]
    X=[]
    for i in humanresults:
        if embed_type=="Raw":
            file_name=i.replace(".mp3","_rawembed.npy")
        elif embed_type=="Normalized":
            file_name=i.replace(".mp3","_embed.npy")
        else:
            raise Exception("ERROR with embed type")
        an_x=np.load(split_path / file_name)
        X.append(an_x)
    return np.array(X)

In [41]:
# params
embed_types=["Raw","Normalized"] # "Raw" or "Normalized"
map_reduce_embeds= ["Average","Concat"]

embed_type=embed_types[0]
map_reduce_type=map_reduce_embeds[0]



In [42]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

all_results={}
for embed_type in embed_types:
    for map_reduce_type in map_reduce_embeds:
        X=pick_embed(embed_type)
        X=map_reduce(X,map_reduce_type)
        print("\n {} embeddings and merged by {} ".format(embed_type,map_reduce_type))
        results=get_results(X,y_true_dict,classifiers,names,tag_set,logs=False)
        all_results[embed_type+"_"+map_reduce_type]=results
        print_result2(results)


 Raw embeddings and merged by Average 
Tag             Songbird    Water Bird    Insect    Running Water    Rain    Cable    Wind    Aircraft
------------  ----------  ------------  --------  ---------------  ------  -------  ------  ----------
N. Neighbors        0.69          0.62      0.76             0.8     0.74     0.8     0.86        0.57
Linear SVM          0.8           0.78      0.87             0.89    0.78     0.82    0.87        0.89
RBF SVM             0.77          0.74      0.8              0.87    0.84     0.87    0.85        0.72
Gaussian P.         0.83          0.78      0.86             0.91    0.85     0.87    0.9         0.94
Decision T.         0.71          0.64      0.76             0.83    0.68     0.75    0.81        0.61
Random F.           0.69          0.68      0.82             0.78    0.81     0.77    0.84        0.91
NN                  0.79          0.76      0.86             0.92    0.83     0.86    0.9         0.97
AdaBoost            0.74         

In [ ]:
# for embed_type in embed_types:
#     for map_reduce_type in map_reduce_embeds:
#         results=all_results[embed_type+"_"+map_reduce_type]
#         print("\n{} embeddings and merged by {} ".format(embed_type,map_reduce_type))
#         print_result2(results)

In [ ]:
# tag="Songbird"
# X_train, X_test, y_train, y_test = train_test_split(
#     X_mean, y_true_dict[tag], test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(
#     X_train, y_train, test_size=0.25, random_state=42)

# for name, clf in zip(names, classifiers):
# #         ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
#     print(name)

#     try:
#         clf.fit(X_train, y_train)
#         y_val_pred=clf.predict_proba(X_val)
#         score=roc_auc_score(y_val,y_val_pred[:,1:])
#         print(y_val_pred[:10,1:])
#         print(clf.classes_)
# #         score = clf.score(X_val, y_val)
#     except:
#         print("ERROR")
        